# SocietyOfMindAgent

This notebook demonstrates the SocietyOfMindAgent, which runs a group chat as an internal monologue, but appears to the external world as a single agent. This confers three distinct advantages:

1. It provides a clean way of producing a hierarchy of agents, hiding complexity as inner monologues.
2. It provides a consistent way of extracting an answer from a lengthy group chat (normally, it is not clear which message is the final response, and the response itself may not always be formatted in a way that makes sense when extracted as a standalone message).
3. It provides a way of recovering when agents exceed their context window constraints (the inner monologue is protected by try-catch blocks)

````{=mdx}
:::info Requirements
Install `pyautogen`:
```bash
pip install pyautogen
```

For more information, please refer to the [installation guide](/docs/installation/).
:::
````

In [1]:
import autogen  # noqa: E402
import os
import random
from autogen.coding import LocalCommandLineCodeExecutor

cache_seed = random.randint(0, 1000)

# Print current working directory
print(f"Current working directory: {os.getcwd()}")
# Print work_dir
print(f"Work directory: {os.path.abspath('./coding')}")
# Create the work directory if it does not exist
os.makedirs("./coding", exist_ok=True)

llm_config = {
    "timeout": 600,
    "cache_seed": cache_seed,  # change the seed for different trials
    "config_list": [
        {
            "model": "NotRequired",
            "api_key": "NotRequired",
            "base_url": "http://0.0.0.0:4000",
            "stream": True
        }
    ],
    "temperature": 0,
}

Current working directory: /Users/jethroestrada/Desktop/External_Projects/GPT/ResumeChatbot/autogen_jet/notebook
Work directory: /Users/jethroestrada/Desktop/External_Projects/GPT/ResumeChatbot/autogen_jet/notebook/coding


````{=mdx}
:::tip
Learn more about configuring LLMs for agents [here](/docs/topics/llm_configuration).
:::
````

### Example Group Chat with Two Agents

In this example, we will use an AssistantAgent and a UserProxy agent (configured for code execution) to work together to solve a problem. Executing code requires *at least* two conversation turns (one to write the code, and one to execute the code). If the code fails, or needs further refinement, then additional turns may also be needed. We will then wrap these agents in a SocietyOfMindAgent, hiding the internal discussion from other agents (though will still appear in the console), and ensuring that the response is suitable as a standalone message.

#### Construct the Inner-Monologue Agents
We begin by constructing the inner-monologue agents. These are the agents that do that real work.

In [2]:
DEFAULT_SYSTEM_MESSAGE = """You are a helpful AI assistant providing python code or shell scripts for user tasks. Ensure the code is executable and complete. Clearly explain your plans and use 'print' for output. Include types and write separate tests for each API route. Add # filename: <filename> at the beginning of the code block to indicate the file name where the code should be saved. Don't include multiple code blocks in one response.
""".strip()

assistant = autogen.AssistantAgent(
    "inner-assistant",
    system_message=DEFAULT_SYSTEM_MESSAGE,
    llm_config=llm_config,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

code_interpreter = autogen.UserProxyAgent(
    "inner-code-interpreter",
    human_input_mode="NEVER",
    code_execution_config={
        # the executor to run the generated code
        "executor": LocalCommandLineCodeExecutor(
            work_dir="./coding",
            execution_policies={
                "bash": True,
                "shell": True,
                "sh": True,
                "pwsh": True,
                "powershell": True,
                "ps1": True,
                "python": True,
                "javascript": True,
                "html": False,
                "css": False,
            }
        )
    },
    default_auto_reply="",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

groupchat = autogen.GroupChat(
    agents=[assistant, code_interpreter],
    messages=[],
    speaker_selection_method="round_robin",  # With two agents, this is equivalent to a 1:1 conversation.
    allow_repeat_speaker=False,
    max_round=8,
)

manager = autogen.GroupChatManager(
    groupchat=groupchat,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
)

#### Construct and Run the SocietyOfMind Agent
We now wrap the inner group-chat with the SocietyOfMind Agent, and create a UserProxy to talk to it.

In [3]:
from autogen.agentchat.contrib.society_of_mind_agent import SocietyOfMindAgent  # noqa: E402

task = "On which days in 2024 was Microsoft Stock higher than $370?"

# Initialize tokens_per_name with a default value
tokens_per_name = {}

# Setup the SocietyOfMindAgent
society_of_mind_agent = SocietyOfMindAgent(
    "society_of_mind",
    chat_manager=manager,
    llm_config=llm_config,
)

# Setup the UserProxyAgent
user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    default_auto_reply="",
    is_termination_msg=lambda x: True,
    code_execution_config={
        # the executor to run the generated code
        "executor": LocalCommandLineCodeExecutor(
            work_dir="./coding",
            execution_policies={
                "bash": True,
                "shell": True,
                "sh": True,
                "pwsh": True,
                "powershell": True,
                "ps1": True,
                "python": True,
                "javascript": True,
                "html": False,
                "css": False,
            }
        )
    },
)

# Start the chat
user_proxy.initiate_chat(society_of_mind_agent, message=task)


user_proxy (to society_of_mind):

On which days in 2024 was Microsoft Stock higher than $370?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
society_of_mind (to chat_manager):

On which days in 2024 was Microsoft Stock higher than $370?

--------------------------------------------------------------------------------

Next speaker: inner-assistant

# filename: msft_stock.py

To solve this problem, we'll use the Yahoo Finance API to fetch historical stock prices for Microsoft (MSFT) and then iterate through each day's data to check if the closing price was higher than $370.

Here's the Python code:

```python
import yfinance as yf
from datetime import datetime

# Define the start and end dates
start_date = '2024-01-01'
end_date = '2024-12-31'

# Fetch historical stock prices for MSFT
msft_data = yf.download('MSFT', start=start_date, end=end_date)

# Initialize an empty list to store the days when the closing price was high

Model llama3 not found. Using cl100k_base encoding.




Model: llama3
Tokens per message: 3
Tokens per name: 1
[autogen.oai.client: 07-27 03:40:10] {329} WARNING - Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
inner-assistant (to chat_manager):

# filename: msft_stock.py

To solve this problem, we'll use the Yahoo Finance API to fetch historical stock prices for Microsoft (MSFT) and then iterate through each day's data to check if the closing price was higher than $370.

Here's the Python code:

```python
import yfinance as yf
from datetime import datetime

# Define the start and end dates
start_date = '2024-01-01'
end_date = '2024-12-31'

# Fetch historical stock prices for MSFT
msft_data = yf.download('MSFT', start=start_date, end=end_date)

# Initialize an empty list to store the days when the closing price was higher than $370
higher_than_370_days = []

# Iterate through each day's data
for index, row in msft_data.i

Model llama3 not found. Using cl100k_base encoding.




Model: llama3
Tokens per message: 3
Tokens per name: 1
[autogen.oai.client: 07-27 03:40:56] {329} WARNING - Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
inner-assistant (to chat_manager):

The error message indicates that the `strptime()` function expects a string as its first argument, but it's receiving a `Timestamp` object instead. This is because the `index` variable in your code is actually a `Timestamp` object, not a string.

To fix this issue, you can convert the `Timestamp` object to a string using the `strftime()` method before passing it to the `strptime()` function. Here's how you can modify your code:

```python
import yfinance as yf
from datetime import datetime

# Define the start and end dates
start_date = '2024-01-01'
end_date = '2024-12-31'

# Fetch historical stock prices for MSFT
msft_data = yf.download('MSFT', start=start_date, end=end_date)

#

...

Model llama3 not found. Using cl100k_base encoding.




Model: llama3
Tokens per message: 3
Tokens per name: 1
[autogen.oai.client: 07-27 03:42:42] {329} WARNING - Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
inner-assistant (to chat_manager):

The code is now running without errors and producing the expected output. It's fetching historical stock prices for Microsoft (MSFT) from Yahoo Finance, then iterating through each day's data to find the days when the closing price was higher than $370. The dates of these events are being printed out.

Here's a breakdown of what the code is doing:

1. Importing necessary libraries: `yfinance` and `datetime`.
2. Defining the start and end dates for which we want to fetch historical stock prices.
3. Fetching the historical stock prices using `yf.download()`.
4. Initializing an empty list to store the days when the closing price was higher than $370.
5. Iterating through each day's

Model llama3 not found. Using cl100k_base encoding.




Model: llama3
Tokens per message: 3
Tokens per name: 1
[autogen.oai.client: 07-27 03:43:02] {329} WARNING - Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
society_of_mind (to user_proxy):



--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'On which days in 2024 was Microsoft Stock higher than $370?', 'role': 'assistant'}, {'content': '', 'role': 'user'}], summary='', cost={'usage_including_cached_inference': {'total_cost': 0, 'llama3': {'cost': 0, 'prompt_tokens': 2841, 'completion_tokens': 0, 'total_tokens': 2841}}, 'usage_excluding_cached_inference': {'total_cost': 0, 'llama3': {'cost': 0, 'prompt_tokens': 2841, 'completion_tokens': 0, 'total_tokens': 2841}}}, human_input=[])

#### Remarks

There are a few things to notice about this output:
- First, the user_proxy sent only one message to the society_of_mind agent, and received only one message in response. As far as it is concerned, the society_of_mind agent is the only agent in the chat.
- Second, the final response is formatted in a way that is standalone. Unlike the prior response, it makes no reference of a previous script or execution, and it lacks the TERMINATE keyword that ended the inner monologue.